## Mapping South African Covid-19 Vaccine Opinions 

### Method Overview
1. Read data from survey
2. data cleaning and preprocessing
    - Geolocation of response
3. Statistical Analysis
4. Sentiment Analysis
5. Discussion

In [1]:
## Import libraries and packages
import snscrape.modules.twitter as sntwitter
import pandas as pd
import numpy as np
import geopandas as gpd

import re
import string
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize,sent_tokenize
import nltk.corpus
from nltk.corpus import stopwords
import neattext.functions as nfx
from pandasql import sqldf
import seaborn as sns
from datetime import datetime
import openpyxl
from textblob import TextBlob
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax


C:\Users\teris\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
# read data 
survey_df = pd.read_csv('Data/Survey_Responses.csv',sep=",")
prolific_df = pd.read_csv('Data/Prolific_Responses.csv',sep=",")

### Data Cleaning and Preprocessing 
Clean Data
* The data exported from Qualitrics has the headers in row 2 and the data starts in row 3


In [3]:
# Replace headers
new_header = prolific_df.iloc[0] #first row for the header
prolific_df = prolific_df[1:] #data less the header row
prolific_df.columns = new_header #set the header row as the df header
prolific_df = prolific_df[1:] #data less the header row
prolific_df
new_header = survey_df.iloc[0] #first row for the header
survey_df = survey_df[1:] # data less the header row
survey_df.columns = new_header #set the header row as the df header
survey_df = survey_df[1:] #data less the header row
survey_df

,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Do you know more people who have taken the vaccine or more people that have not taken the vaccine?,"What is your main source of trusted news and/or information? - Selected Choice - News channels, eg. eNCA",What is your main source of trusted news and/or information? - Selected Choice - Newspapers,What is your main source of trusted news and/or information? - Selected Choice - YouTube,What is your main source of trusted news and/or information? - Selected Choice - Facebook or other social media,What is your main source of trusted news and/or information? - Selected Choice - Government,What is your main source of trusted news and/or information? - Selected Choice - Scientific research papers,What is your main source of trusted news and/or information? - Selected Choice - Other,What is your main source of trusted news and/or information? - Other - Text,Comments
2,2022-06-07 10:55:18,2022-06-07 10:57:39,IP Address,137.215.99.2,100,141,True,2022-06-07 10:57:40,R_yseNcThgGNVwT73,NaN,...,More people who have been vaccinated,"News channels, eg. eNCA",NaN,NaN,Facebook or other social media,Government,NaN,NaN,NaN,NaN
3,2022-06-07 10:59:28,2022-06-07 11:01:49,IP Address,85.104.55.182,100,140,True,2022-06-07 11:01:49,R_2ypzw38J2XQjvwL,NaN,...,More people who have been vaccinated,NaN,NaN,NaN,NaN,NaN,Scientific research papers,NaN,NaN,NaN
4,2022-06-07 10:59:23,2022-06-07 11:02:05,IP Address,105.186.249.163,100,162,True,2022-06-07 11:02:06,R_10DxZI4X9906ne7,NaN,...,More people who have been vaccinated,"News channels, eg. eNCA",Newspapers,NaN,Facebook or other social media,NaN,Scientific research papers,NaN,NaN,NaN
5,2022-06-07 11:04:09,2022-06-07 11:06:17,IP Address,105.213.179.249,100,128,True,2022-06-07 11:06:17,R_3MAqPKaRYkZPDNX,NaN,...,More people who have not been vaccinated,NaN,NaN,NaN,NaN,NaN,NaN,Other,none.,NaN
6,2022-06-07 11:05:23,2022-06-07 11:08:24,IP Address,41.198.134.163,100,180,True,2022-06-07 11:08:24,R_3Rn49K8zsssarq7,NaN,...,More people who have been vaccinated,"News channels, eg. eNCA",NaN,YouTube,NaN,NaN,Scientific research papers,Other,"My lecturers from Anatomy, physiology, etc",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,2022-06-26 23:06:29,2022-06-26 23:13:02,IP Address,41.198.134.163,100,393,True,2022-06-26 23:13:03,R_3kiX3UhbjtMZiiC,NaN,...,More people who have been vaccinated,NaN,NaN,YouTube,Facebook or other social media,NaN,NaN,NaN,NaN,NaN
340,2022-06-21 20:59:17,2022-06-21 20:59:39,IP Address,209.203.22.31,8,21,False,2022-06-28 08:53:43,R_3nH6dhWXgLvZ7oD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
341,2022-06-21 10:29:42,2022-06-21 10:49:25,IP Address,41.114.1.169,58,1183,False,2022-06-28 08:53:46,R_1IT2r7sd8yyMNB0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,2022-06-21 10:39:44,2022-06-21 10:43:08,IP Address,196.41.10.34,88,204,False,2022-06-28 08:53:48,R_12EfL2ISf755elz,NaN,...,More people who have been vaccinated,NaN,NaN,NaN,NaN,NaN,NaN,Other,All forms of media,NaN


Not all responses can be used, remove those that could not be used. 
Responses that are were removed included:
- survey type of 'Survey Preview'
- those who did not agree to the survey
- not finished

In [4]:
# Drop rows with survey previews
survey_df = survey_df.drop(survey_df.index[survey_df['Response Type'] == "Survey Preview"])
prolific_df = prolific_df.drop(prolific_df.index[prolific_df['Response Type'] == "Survey Preview"])
# reset index
survey_df = survey_df.reset_index().drop(columns=['index'])
prolific_df = prolific_df.reset_index().drop(columns=['index'])


In [5]:
# Drop rows which participants did not consent to the survey
survey_df = survey_df.drop(survey_df.index[survey_df['Do you agree to participate in this survey?'] != "Agree"])
prolific_df = prolific_df.drop(prolific_df.index[prolific_df['Do you agree to participate in this survey?'] != "Agree"])

# reset index
survey_df = survey_df.reset_index().drop(columns=['index'])
prolific_df = prolific_df.reset_index().drop(columns=['index'])

In [6]:
#drop rows that were incompleted
survey_df = survey_df.drop(survey_df.index[survey_df['Finished'] != "True"])
prolific_df = prolific_df.drop(prolific_df.index[prolific_df['Finished'] != "True"])

# reset index
survey_df = survey_df.reset_index().drop(columns=['index'])
prolific_df = prolific_df.reset_index().drop(columns=['index'])

In [7]:
#drop rows that participants said that did not have a South African nationality
survey_df = survey_df.drop(survey_df.index[survey_df['What is your Nationality? - Selected Choice'] != "South Africa"])
prolific_df = prolific_df.drop(prolific_df.index[prolific_df['What is your Nationality? - Selected Choice'] != "South Africa"])
prolific_df = prolific_df.reset_index().drop(columns=['index'])
survey_df = survey_df.reset_index().drop(columns=['index'])


In [8]:
# drop rows whcih have people who don't live in South Africa
prolific_df = prolific_df.drop(prolific_df.index[prolific_df['Do you currently reside in South Africa?'] == "No"])
prolific_df = prolific_df.reset_index().drop(columns=['index'])


Merge data 

In [9]:
frames = [survey_df, prolific_df]
df = pd.concat(frames)

In [10]:
# Drop columns 
df = df.drop(columns=['Start Date', 'End Date',  'IP Address', 'Progress',
       'Duration (in seconds)',  'Response ID',
       'Recipient Last Name', 'Recipient First Name', 'Recipient Email',
       'External Data Reference', 
       'Distribution Channel','User Language',
       'What is your Prolific ID?\n\nPlease note that this response should auto-fill with the correct ID',
       'PROLIFIC_PID'
])
# reset index
df = df.reset_index().drop(columns=['index'])


#### Geolocate responses
Preprocess location information

In [11]:
df['Origin'] = df['Which city/town are you originally from?'].str.lower()
df['Current'] = df['Which city/town do you currently live in?'].str.lower()
df['Origin'] = df['Origin'].str.title()
df['Current'] = df['Current'].str.title()
df['Origin'] = df['Origin'].str.strip()
df['Current'] = df['Current'].str.strip()
df['Origin'] = df['Origin'].astype(str) + ',South Africa'
df['Current'] = df['Current'].astype(str) + ',South Africa'

In [12]:
#### Repair locations

df["Current"] = df["Current"].replace(['Vanderbiljpark,South Africa'],'Vanderbijlpark,South Africa')
df["Current"] = df["Current"].replace(['Rusternburg,South Africa'],'Rustenburg,South Africa')
df["Current"] = df["Current"].replace(["Richard'S Bay,South Africa"],'Richards Bay,South Africa')
df["Current"] = df["Current"].replace(["Pta,South Africa"],'Pretoria,South Africa')
df["Current"] = df["Current"].replace(["Pre.Toria,South Africa"],'Pretoria,South Africa')
df["Current"] = df["Current"].replace(["Mafikeng,South Africa"],'Mahikeng,South Africa')
df["Current"] = df["Current"].replace(["Johannesburg South,South Africa"],'Johannesburg,South Africa')
df["Current"] = df["Current"].replace(["Johanessburg,South Africa"],'Johannesburg,South Africa')
df["Current"] = df["Current"].replace(["Joburg South,South Africa"],'Johannesburg,South Africa')
df["Current"] = df["Current"].replace(["Jhb/Northcliff,South Africa"],'Johannesburg,South Africa')
df["Current"] = df["Current"].replace(["Johanessberg South,South Africa"],'Johannesburg,South Africa')
df["Current"] = df["Current"].replace(["Hatfirld,South Africa"],'Hatfield,South Africa')
df["Current"] = df["Current"].replace(["Ekurhulen,South Africa"],'Ekurhuleni,South Africa')
df["Current"] = df["Current"].replace(["Cape  Town,South Africa"],'Cape Town,South Africa')
df["Current"] = df["Current"].replace(["Johanessburg,South Africa"],'Johannesburg,South Africa')
df["Current"] = df["Current"].replace(["Capetown,South Africa"],'Cape Town,South Africa')
df["Current"] = df["Current"].replace(["Hamanskraal,South Africa"],'Hammanskraal,South Africa')
df["Current"] = df["Current"].replace(["Johburg,South Africa"],'Johannesburg,South Africa')
df["Current"] = df["Current"].replace(['Pe,South Africa'],'Port Elizabeth,South Africa')
df["Current"] = df["Current"].replace(['Johanessberg,South Africa'],'Johannesburg,South Africa')

df["Origin"] = df["Origin"].replace(["Johannesburg South,South Africa"],'Johannesburg,South Africa')
df["Origin"] = df["Origin"].replace(['As Above,South Africa'],'Vanderbijlpark,South Africa')
df["Origin"] = df["Origin"].replace(['Pietermaritzburb,South Africa'],'Pietermaritzburg,South Africa')
df["Origin"] = df["Origin"].replace(["Mount Frere,South Africa"],'KwaBhaca,South Africa')
df["Origin"] = df["Origin"].replace(["Koue Bokkeveld,South Africa"],'Bokkeveld,South Africa')
df["Origin"] = df["Origin"].replace(["Rusternburg,South Africa"],'Rustenburg,South Africa')
df["Origin"] = df["Origin"].replace(["Roodpoort,South Africa"],'Roodepoort,South Africa')
df["Origin"] = df["Origin"].replace(["Johanessberg,South Africa"],'Johannesburg,South Africa')
df["Origin"] = df["Origin"].replace(["Thohoyoandou,South Africa"],'Thohoyandou,South Africa')
df["Origin"] = df["Origin"].replace(["Emzinto,South Africa"],'Umzinto,South Africa')
df["Origin"] = df["Origin"].replace(["Venda,South Africa"],'Limpopo,South Africa')
df["Origin"] = df["Origin"].replace(["Venda,South Africa"],'Limpopo,South Africa')
df["Origin"] = df["Origin"].replace(['Pe,South Africa'],'Port Elizabeth,South Africa')
df["Origin"] = df["Origin"].replace(["Saselamani,South Africa"],'Malamulele,South Africa')


### Geolocation using Nominatim

In [15]:
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
mapbox_token ='pk.eyJ1IjoiY3Jhenl0ZXJpIiwiYSI6ImNsNHM2N2Z3aDBkNHczZG82MGVramJvY2oifQ.Re9w4pfsIMc_UPA4k1rs3w'
import plotly.express as px
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
from geopy.exc import GeocoderTimedOut

In [16]:
# Origin 
import time
geolocator = Nominatim(user_agent="geoapiExercises", timeout= time.sleep(1.1))
    
count = 0
df['Olat'] =''
df['Olng'] = ''
for row in df['Origin']:
    # print(row)
    location = geolocator.geocode(row)
    if location is None:
        df['Olat'][count] = '0'
        df['Olng'][count] = '0'
    else:
        df['Olat'][count] = location.latitude
        df['Olng'][count] = location.longitude
    count += 1


In [17]:
# Current
count = 0
df['Clat'] =''
df['Clng'] = ''
for row in df['Current']:
    # print(row)
    location = geolocator.geocode(row)
    if location is None:
        df['Clat'][count] = '0'
        df['Clng'][count] = '0'
    else:
        df['Clat'][count] = location.latitude
        df['Clng'][count] = location.longitude
    count += 1

In [ ]:
# convert lat/lon to floats
df.Clng = df.Clng.astype('float')
df.Clat = df.Clat.astype('float')
df.Olng = df.Olng.astype('float')
df.Olat = df.Olat.astype('float')

#### check for locations not geolocated


In [ ]:
df[df.Clng<=0]


In [ ]:
df[df.Clat<=0]


In [ ]:
df[df.Clng>=0]


In [ ]:
df[df.Clat>=0]


In [ ]:
df[df.Olng<=0]


In [ ]:
df[df.Olat<=0]


In [ ]:
df[df.Olng>=0]


In [ ]:
df[df.Olat>=0]

#### drop the unmapped rows


In [ ]:

df = df[df.Olng!=0]
df = df[df.Clng!=-2.49020920207643]
df = df.reset_index().drop(columns=['index'])

In [ ]:
df = df[df.Olat!=5]
df = df.reset_index().drop(columns=['index'])

## Query Data

In [ ]:
list(df)

In [ ]:
q = 'SELECT count(*), "How old are you?" FROM df group by "How old are you?"'

sqldf(q, globals())

In [ ]:
df.loc[(df["How old are you?"] == '`20')]
df["How old are you?"] = df["How old are you?"].replace(['`20'],'20')

In [ ]:
df["How old are you?"] = df["How old are you?"].replace(['54 years'],'54')

In [ ]:
q = 'SELECT SUM(CASE WHEN "How old are you?" < 18 THEN 1 ELSE 0 END ) AS [UNDER 18],SUM(CASE WHEN "How old are you?" BETWEEN 18 AND 24 THEN 1 ELSE 0 END ) AS [18-24],SUM(CASE WHEN "How old are you?" BETWEEN 25 AND 34 THEN 1 ELSE 0 END ) AS [25-34] ,SUM(CASE WHEN "How old are you?" BETWEEN 35 AND 54 THEN 1 ELSE 0 END ) AS [35-54] ,SUM(CASE WHEN "How old are you?" BETWEEN 55 AND 64 THEN 1 ELSE 0 END ) AS [55-64] ,SUM(CASE WHEN "How old are you?" > 64 THEN 1 ELSE 0 END ) AS [OVER 65], SUM(CASE WHEN "How old are you?" LIKE NULL THEN 1 ELSE 0 END ) AS [NULL] FROM df'

sqldf(q, globals())

In [ ]:
q = 'SELECT count(*), "Have you taken the Covid-19 vaccine?" FROM df group by"Have you taken the Covid-19 vaccine?"  '

sqldf(q, globals())

In [ ]:
q = 'SELECT count(*), "What is your main source of trusted news and/or information? - Selected Choice - News channels, eg. eNCA" as "News" FROM df group by "What is your main source of trusted news and/or information? - Selected Choice - News channels, eg. eNCA"  '

sqldf(q, globals())

In [ ]:
q = 'SELECT  "What is your highest level of education? - Selected Choice", count(*) FROM df group by "What is your highest level of education? - Selected Choice"'

sqldf(q, globals())

In [ ]:
q = 'SELECT  "What is your highest level of education? - Other - Text", count(*) FROM df group by "What is your highest level of education? - Other - Text"'

sqldf(q, globals())

In [ ]:
df["What is your highest level of education? - Other - Text"] = df["What is your highest level of education? - Other - Text"].replace(['54 years'],'54')

In [ ]:
df.to_excel('D:/2022/October 2022/Data/all.xlsx')

## Statistical Analysis

In [ ]:
#%pip install sodapy
from sodapy import Socrata
# for basemaps
import contextily as ctx
# For spatial statistics
import esda
from esda.moran import Moran, Moran_Local
import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation
import libpysal as lps
# Graphics
import plotly.express as px


In [ ]:
# province data
gdf = gpd.read_file('D:/2022/Research Assignment/August 2022/Data/provinces.shp')

In [ ]:
# show first 5 rows
gdf.head()

In [ ]:
# show columns and data types
gdf.info()

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
gdf = gdf.to_crs(epsg=4326)